In [68]:
from __future__ import annotations
import random as rd


In [65]:
class Individuo:
    def __init__(self):
        self.x = gerar_bits(13)
        self.y = gerar_bits(13)
        self.fitness = self.calcula_fitness()
# Primeiro vou fazer o init declarando as variáveis x e y e fitness.
# vou usar o rand para gerar bits em uma função separada
    def calcula_fitness(self):
        x_real = self.bits_para_real(self.x)
        y_real = self.bits_para_real(self.y)
        return -x_real**2 - y_real**2 + 6*x_real + 4*y_real

    def bits_para_real(self, bits): ## transformar bits em real
        inteiro = int("".join(map(str, bits)), 2)
        return inteiro / 1000

class Populacao:
    def __init__(self, tamanho_populacao):
        self.tamanho_populacao = tamanho_populacao
        self.populacao = [Individuo() for _ in range(tamanho_populacao)] ## gerador automático de população.

    def ordenar_por_fitness(self):
      self.populacao.sort(key=lambda ind: ind.fitness, reverse=True) ## para fazer elitismo preciso selecionar os melhores de cada geração, para isso preciso dar 'sort'

    def selecionar_pais(self, proporcao_elite=0.05):
        ## aqui eu seleciono os melhores 5% — na apostila dizia entre 2 e 5%
        ## acredito que ao selecionar 5% melhora a dispersibilidade da minha população.
      self.ordenar_por_fitness()  ## uso a função de sort para poder pegar os 5% melhores
      n = max(2, int(self.tamanho_populacao * proporcao_elite))
      ## max(2, ...) garante pelo menos 2 pais, evitando erro que estava dando de só ter 1 pai dependendo do tamanho (5% de 19 por exemplo ...)
      ## tentei fazer com max(1, ...) mas se tiver só 1 pai a população acaba se clonando
      return self.populacao[:n]  ## slice da população elite

def cruzamento(pai1, pai2):
    # Ponto fixo de cruzamento: 6 bits do pai1 + 7 bits do pai2 ou mãe
    pontoCruz = len(pai1.x) - 7

    filhox = pai1.x[:pontoCruz] + pai2.x[pontoCruz:]
    filhoy = pai1.y[:pontoCruz] + pai2.y[pontoCruz:]

    return filhox, filhoy

## agora vou fazer a de mutação, ela vai ter 1% de chance de mutar.
def mutacao(individuo, taxa_mutacao=0.01):
    for i in range(len(individuo.x)):
        if rd.random() < taxa_mutacao:
            individuo.x[i] = 1 - individuo.x[i]
    for i in range(len(individuo.y)):
        if rd.random() < taxa_mutacao:
            individuo.y[i] = 1 - individuo.y[i]






def gerar_bits(n_bits=13):
  numero = rd.getrandbits(n_bits)
  bin_str = format(numero, f'0{n_bits}b')  # ex: '0101011101101'
  return [int(bit) for bit in bin_str]     # lista: [0, 1, 0, 1 ... eu não sei se está certo, mas acredito que a list funciona.
## Pensei em colocar dentro do individuo, mas como é uma função que só vamos usar na primeira geração acredito seria perda de memória colocar dentro da classe

def nova_geracao(pop, taxa_mutacao=0.03, proporcao_elite=0.05):
    elite = pop.selecionar_pais(proporcao_elite)
    nova_pop = elite.copy()

    while len(nova_pop) < pop.tamanho_populacao:
        pai1, pai2 = rd.sample(elite, 2)
        filhox, filhoy = cruzamento(pai1, pai2)

        filho = Individuo()
        filho.x = filhox
        filho.y = filhoy
        mutacao(filho, taxa_mutacao)
        filho.fitness = filho.calcula_fitness()

        nova_pop.append(filho)

    pop.populacao = nova_pop




def main():
    pop = Populacao(1000)
    geracoes = 50

    for g in range(geracoes):
        melhor = pop.populacao[0]
        for individuo in pop.populacao:
            if individuo.fitness > melhor.fitness:
                melhor = individuo
        print(f"Geração {g}: Melhor fitness = {melhor.fitness:.4f} | x = {melhor.bits_para_real(melhor.x):.3f}, y = {melhor.bits_para_real(melhor.y):.3f}")

        if abs(melhor.fitness - 13.000) < 1e-4: ## tive que adicionar redundâcia, depois de vários testes falho mesmo dando 13, descobri que pode ter pequenas diferenças e por isso precisa de redundância.
            print(f"Solução ótima encontrada. fitness: {melhor.fitness:.4f}")
            break

        nova_geracao(pop, taxa_mutacao=0.01)






In [72]:
main()

Geração 0: Melhor fitness = 12.9973 | x = 3.030, y = 1.958
Geração 1: Melhor fitness = 12.9998 | x = 3.006, y = 1.989
Geração 2: Melhor fitness = 12.9999 | x = 3.010, y = 2.001
Geração 3: Melhor fitness = 13.0000 | x = 2.998, y = 2.005
Solução ótima encontrada. fitness: 13.0000
